In [1]:
import numpy as np
import pandas as pd

In [2]:
column_names = ["ID", "weight", "batch"]
df = pd.read_csv("source/euroweight.dat.txt", sep='\t', names=column_names, index_col=0)
df

,weight,batch
ID,,
1,7.512,1
2,7.502,1
3,7.461,1
4,7.562,1
5,7.528,1
...,...,...
1996,7.514,8
1997,7.519,8
1998,7.606,8


In [3]:
weights_by_batches = {}
for batch_number in range(min(df['batch']), max(df['batch']) + 1):
    weights = df[df['batch'] == batch_number]['weight']
    weights_by_batches[batch_number] = np.array(weights)

In [4]:
weights_by_batches

{1: array([7.512, 7.502, 7.461, 7.562, 7.528, 7.459, 7.518, 7.537, 7.517,
        7.605, 7.567, 7.551, 7.486, 7.528, 7.525, 7.581, 7.497, 7.548,
        7.553, 7.544, 7.468, 7.539, 7.537, 7.554, 7.455, 7.459, 7.534,
        7.467, 7.504, 7.508, 7.492, 7.466, 7.477, 7.504, 7.517, 7.506,
        7.543, 7.524, 7.543, 7.505, 7.504, 7.574, 7.548, 7.527, 7.535,
        7.537, 7.55 , 7.49 , 7.487, 7.464, 7.514, 7.504, 7.49 , 7.514,
        7.474, 7.512, 7.492, 7.486, 7.565, 7.58 , 7.561, 7.511, 7.51 ,
        7.482, 7.502, 7.54 , 7.553, 7.554, 7.513, 7.536, 7.536, 7.543,
        7.594, 7.571, 7.51 , 7.542, 7.548, 7.531, 7.509, 7.535, 7.547,
        7.549, 7.512, 7.554, 7.489, 7.553, 7.515, 7.531, 7.538, 7.453,
        7.493, 7.537, 7.485, 7.5  , 7.478, 7.545, 7.506, 7.566, 7.595,
        7.567, 7.521, 7.521, 7.538, 7.531, 7.506, 7.484, 7.505, 7.535,
        7.517, 7.53 , 7.514, 7.526, 7.457, 7.485, 7.52 , 7.498, 7.49 ,
        7.581, 7.486, 7.543, 7.481, 7.491, 7.572, 7.499, 7.531, 7.523,
   

Проверим попарное равенство дисперсий критерием Фишера

In [9]:
from scipy.stats import bartlett, f

results = []

for key1, sample1 in weights_by_batches.items():
    for key2, sample2 in weights_by_batches.items():
        if key1 < key2:
            var1 = np.var(sample1, ddof=1)  # ddof=1 для вычисления выборочной дисперсии
            var2 = np.var(sample2, ddof=1)
            F = var1 / var2

            alpha = 0.05

            n = len(sample1) - 1
            m = len(sample2) - 1

            critical_value = f.ppf(1 - alpha / 2, n, m)
            p_value = 2 * min(f.cdf(F, n, m), 1 - f.cdf(F, n, m))

            results.append((key1, key2, F, critical_value, p_value))

for key1, key2, F, critical_value, p_value in results:
    print(f"Тест Фишера между {key1} и {key2}:")
    if F < critical_value:
        print("Равенство дисперсий не отвергается, p-value =", p_value)
    else:
        print("Равенство дисперсий отвергается, p-value =", p_value)

Тест Фишера между 1 и 2:
Равенство дисперсий не отвергается, p-value = 0.6118309238755042
Тест Фишера между 1 и 3:
Равенство дисперсий не отвергается, p-value = 0.2367495429959407
Тест Фишера между 1 и 4:
Равенство дисперсий отвергается, p-value = 0.01450801877013963
Тест Фишера между 1 и 5:
Равенство дисперсий отвергается, p-value = 0.01961600590987156
Тест Фишера между 1 и 6:
Равенство дисперсий не отвергается, p-value = 0.6628618477236468
Тест Фишера между 1 и 7:
Равенство дисперсий не отвергается, p-value = 0.5235506707343982
Тест Фишера между 1 и 8:
Равенство дисперсий не отвергается, p-value = 0.3695394971253173
Тест Фишера между 2 и 3:
Равенство дисперсий не отвергается, p-value = 0.49897947671015436
Тест Фишера между 2 и 4:
Равенство дисперсий отвергается, p-value = 0.0031958714823918655
Тест Фишера между 2 и 5:
Равенство дисперсий отвергается, p-value = 0.004543751772420057
Тест Фишера между 2 и 6:
Равенство дисперсий не отвергается, p-value = 0.34556969109647806
Тест Фишера м

Проверим равенство всех дисперсий

In [10]:
all_samples = list(weights_by_batches.values())
statistic, p_value = bartlett(*all_samples)

print("Общий Бартлетт-тест для всех выборок:")
if p_value < 0.05:
    print("Равенство дисперсий отвергается для всех выборок, p-value =", p_value)
else:
    print("Равенство дисперсий не отвергается для всех выборок, p-value =", p_value)

Общий Бартлетт-тест для всех выборок:
Равенство дисперсий отвергается для всех выборок, p-value = 0.0005922420870263893


Раз равенство дисперсий не принято, будем проверять средние значения для тех пар выборок, у которых они равны:

In [11]:
from scipy.stats import ttest_ind

mean_results = []

for key1, key2, F, critical_value, p_value in results:
    if F < critical_value:
        sample1 = weights_by_batches[key1]
        sample2 = weights_by_batches[key2]
        t_stat, t_p_value = ttest_ind(sample1, sample2)
        mean_results.append((key1, key2, t_stat, t_p_value))

for key1, key2, t_stat, t_p_value in mean_results:
    print(f"t-тест для средних между {key1} и {key2}:")
    if t_p_value < 0.05:
        print("Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value =", t_p_value)
    else:
        print("Средние значения выборок схожи (не отвергаем нулевую гипотезу), p-value =", t_p_value)

t-тест для средних между 1 и 2:
Средние значения выборок схожи (не отвергаем нулевую гипотезу), p-value = 0.26147780679017946
t-тест для средних между 1 и 3:
Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value = 0.001648507542831034
t-тест для средних между 1 и 6:
Средние значения выборок схожи (не отвергаем нулевую гипотезу), p-value = 0.1458601194945584
t-тест для средних между 1 и 7:
Средние значения выборок схожи (не отвергаем нулевую гипотезу), p-value = 0.26532250649811384
t-тест для средних между 1 и 8:
Средние значения выборок схожи (не отвергаем нулевую гипотезу), p-value = 0.3566197335516895
t-тест для средних между 2 и 3:
Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value = 3.169999483351773e-05
t-тест для средних между 2 и 6:
Средние значения выборок различаются (отвергаем нулевую гипотезу), p-value = 0.010416963435836283
t-тест для средних между 2 и 7:
Средние значения выборок схожи (не отвергаем нулевую гипотезу), p-value = 0

Равенство всех средних не проверяем, т.к. равенство всех дисперсий не выполняется.